# Euphoria Twitter Analysis

## Data Cleaning and Location Geocoding

### Scope: This notebook will show how I combine all my data into one DataFrame, clean the data, and use the HERE API to extract the country and countrycodes of each tweet that has user specified location within it. 

In [38]:
# Import Libraries 
import pandas as pd
import os
import glob
import re
import requests
import json

In [39]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
pwd

'/Users/iizokun/Desktop/ds_programming/euphoria'

In [3]:
join= os.path.join("euphoria*.csv")

In [4]:
join_list = glob.glob(join)

In [5]:
join_list

['euphoria02.csv',
 'euphoria03.csv',
 'euphoria01.csv',
 'euphoria04.csv',
 'euphoria05.csv',
 'euphoria07.csv',
 'euphoria06.csv']

In [6]:
all_tweets = []
for tweet_df in join_list:
    print(f'Loading {tweet_df}')
    
    new_df = pd.read_csv(tweet_df)
    
    all_tweets.append(new_df)

Loading euphoria02.csv
Loading euphoria03.csv
Loading euphoria01.csv


/Users/iizokun/opt/miniconda3/envs/ds102/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (0,10,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Loading euphoria04.csv
Loading euphoria05.csv
Loading euphoria07.csv
Loading euphoria06.csv


In [7]:
len(all_tweets)

7

In [8]:
df = pd.concat(all_tweets)

In [9]:

df.shape

(236640, 22)

In [10]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,0,Really want a fez / Lexi parallel scene to whe...,2022-01-23 23:59:50,1.485402e+18,Katy,katykitty628,1.471191e+18,NaN,NaN,just a girl in a man's world 😌. ...,...,82.0,556.0,113.0,0.0,2021-12-15 18:51:50,https://pbs.twimg.com/profile_images/147119180...,True,False,0.0,0.0
1,1,#Euphoria I hope Maddy pulls the hair out that...,2022-01-23 23:59:33,1.485402e+18,𝐕𝐢𝐫𝐠𝐞𝐚𝐮𝐱,_njauu,9.510643e+17,here,NaN,"you're in my business, don't do that.",...,221.0,155447.0,45787.0,1.0,2018-01-10 12:12:53,https://pbs.twimg.com/profile_images/147736305...,True,False,0.0,0.0
2,2,Between #AndJustLikeThat and #Euphoria @HBO is...,2022-01-23 23:59:04,1.485402e+18,Steph Swinton,StephSwinton,2.390617e+07,New York,NaN,Just a girl from the city.\nalt Twitter: TheAr...,...,729.0,32677.0,28747.0,5.0,2009-03-12 05:32:39,https://pbs.twimg.com/profile_images/950198226...,False,False,0.0,2.0
3,3,its #Euphoria day!!!!,2022-01-23 23:59:04,1.485402e+18,felic-titty,lilbootybbg,1.173761e+18,in therapy,NaN,she/her 21 :),...,145.0,42553.0,3027.0,0.0,2019-09-17 00:52:26,https://pbs.twimg.com/profile_images/145893433...,True,False,0.0,2.0
4,4,YESSSIR DOING A QUICK STREAM BEFOE TONIGHT'S #...,2022-01-23 23:58:56,1.485402e+18,Chris RossVlogs,ChrisRossvlogs,1.347083e+18,Mr. Hollywood,https://t.co/3xdQXI2Zv8,"Actor/Director, & Youtuber Use code “ChrisRoss...",...,276.0,4211.0,4099.0,1.0,2021-01-07 07:32:01,https://pbs.twimg.com/profile_images/141594587...,True,False,1.0,3.0


In [11]:
df = df.drop(columns = ['Unnamed: 0'])

In [12]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
16976,I hate that I have a gut feeling that somethin...,2022-02-15 00:00:31,1.493375e+18,olivia,Liv_Jord,1.278210e+18,NaN,NaN,21~NC make today count she/her,False,...,367.0,12109.0,2144.0,0.0,2020-07-01 06:12:49,https://pbs.twimg.com/profile_images/141830766...,True,False,0.0,1.0
16977,What’s with all the adults on Euphoria just ca...,2022-02-15 00:00:21,1.493375e+18,(toe•lu)🇳🇬,skiipTOmyLU,2.456662e+07,"Houston, TX",NaN,"I came, I saw, I left early.",False,...,366.0,8476.0,23075.0,7.0,2009-03-15 18:56:40,https://pbs.twimg.com/profile_images/149364342...,False,False,0.0,0.0
16978,“Episode eight is where we’ll get that sense o...,2022-02-15 00:00:20,1.493375e+18,GRAZIA,graziatweets,2.041904e+07,New York,https://t.co/pifa2NBobd,The number one destination for smart and styli...,True,...,794.0,936.0,7336.0,357.0,2009-02-09 06:02:12,https://pbs.twimg.com/profile_images/132316780...,False,False,1.0,0.0
16979,Rue be like #Euphoria https://t.co/E0Z6AEYyEb,2022-02-15 00:00:15,1.493375e+18,the business bitch,_babuba_,4.765358e+09,NaN,NaN,πίβοτ σε εξέλ και παρκέ,False,...,335.0,83433.0,36282.0,14.0,2016-01-16 00:16:14,https://pbs.twimg.com/profile_images/148817450...,False,False,0.0,1.0
16980,Unique Rotating Ultra-Thin Steel Watch👉https:/...,2022-02-15 00:00:01,1.493375e+18,Lucky Super Store,LSS_Store,1.062288e+18,"715 Main St, Asbury Park, NJ",https://t.co/UTYsdnPioj,NaN,False,...,1477.0,4279.0,5252.0,2.0,2018-11-13 10:15:07,https://pbs.twimg.com/profile_images/146425578...,False,False,0.0,0.0


In [13]:
df.sample(5)


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
24837,Oh okay so the first minutes of the new #Eupho...,2022-02-08 00:13:59,1.490841e+18,Chelsey 🌜,ChelseyWithAWhy,1.284052e+08,New Jersey,https://t.co/fYHqSk4CT5,What a thing to be human 🥀🌿 Chaotic tweeter ✨K...,False,...,490.0,92631.0,9008.0,9.0,2010-04-01 01:23:25,https://pbs.twimg.com/profile_images/142995140...,False,False,0.0,1.0
10474,I feel like I’m the only person that hasn’t wa...,2022-02-09 19:56:17,1.491501e+18,Cherubin Mukinay,cheebe_king,3.473412e+08,"Glasgow, Scotland",NaN,Snapchat- cmpunk434-xx Instagram- cherubinmuki...,False,...,5001.0,306320.0,362117.0,78.0,2011-08-02 17:05:45,https://pbs.twimg.com/profile_images/134504568...,False,False,0.0,0.0
5006,I've just watched episode S02 | E08 of Euphori...,2022-02-28 20:14:44,1.498391e+18,thiago,newaxizs,1.477470e+18,loonaverse,https://t.co/btSd9NhNUa,dead poets society (1989) dir. peter weir,False,...,113.0,748.0,933.0,0.0,2022-01-02 02:39:43,https://pbs.twimg.com/profile_images/149691204...,True,False,0.0,0.0
36542,someone had to do it so yw #EuphoriaHBOMax #Eu...,2022-02-28 05:33:36,1.498169e+18,me:,39thRaikage,9.878557e+17,The Trenches,NaN,CEO of Nobody Cares Inc. | ig: @ sillowet,False,...,54.0,851.0,108.0,0.0,2018-04-22 00:48:58,https://pbs.twimg.com/profile_images/133867281...,True,False,0.0,1.0
4471,Lol @ Nate sending his dad to prison because L...,2022-02-28 20:37:25,1.498397e+18,A. Skywalker,Walkerofthewild,7.643828e+08,Neon Valley Street,NaN,"#KeybladeMaster Born in the city, bred in the...",False,...,443.0,1338.0,12637.0,1.0,2012-08-17 20:23:25,https://pbs.twimg.com/profile_images/100441035...,False,False,0.0,3.0


Next Step would be to fill in the column names

In [14]:
df.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20'],
      dtype='object')

In [15]:
df.columns = ['tweet_full_text', 'tweet_created_at','tweet_id', 'tweet_username','tweet_user_screename','tweet_userid_str',
                   'tweet_user_location','tweet_user_url', 'tweet_user_description', 'tweet_user_verified','tweet_user_followers_count',
                   'tweet_user_friends_count', 'tweet_user_favorites_count', 'tweet_user_statuses_count','tweet_user_listed_count',
                   'tweet_user_created_at', 'user_profile_image_url', 'user_default_profile','user_default_profile_image', 'tweet_retweet_count', 'tweet_favorite_count']

In [16]:
df.head()

,tweet_full_text,tweet_created_at,tweet_id,tweet_username,tweet_user_screename,tweet_userid_str,tweet_user_location,tweet_user_url,tweet_user_description,tweet_user_verified,...,tweet_user_friends_count,tweet_user_favorites_count,tweet_user_statuses_count,tweet_user_listed_count,tweet_user_created_at,user_profile_image_url,user_default_profile,user_default_profile_image,tweet_retweet_count,tweet_favorite_count
0,Really want a fez / Lexi parallel scene to whe...,2022-01-23 23:59:50,1.485402e+18,Katy,katykitty628,1.471191e+18,NaN,NaN,just a girl in a man's world 😌. ...,False,...,82.0,556.0,113.0,0.0,2021-12-15 18:51:50,https://pbs.twimg.com/profile_images/147119180...,True,False,0.0,0.0
1,#Euphoria I hope Maddy pulls the hair out that...,2022-01-23 23:59:33,1.485402e+18,𝐕𝐢𝐫𝐠𝐞𝐚𝐮𝐱,_njauu,9.510643e+17,here,NaN,"you're in my business, don't do that.",False,...,221.0,155447.0,45787.0,1.0,2018-01-10 12:12:53,https://pbs.twimg.com/profile_images/147736305...,True,False,0.0,0.0
2,Between #AndJustLikeThat and #Euphoria @HBO is...,2022-01-23 23:59:04,1.485402e+18,Steph Swinton,StephSwinton,2.390617e+07,New York,NaN,Just a girl from the city.\nalt Twitter: TheAr...,False,...,729.0,32677.0,28747.0,5.0,2009-03-12 05:32:39,https://pbs.twimg.com/profile_images/950198226...,False,False,0.0,2.0
3,its #Euphoria day!!!!,2022-01-23 23:59:04,1.485402e+18,felic-titty,lilbootybbg,1.173761e+18,in therapy,NaN,she/her 21 :),False,...,145.0,42553.0,3027.0,0.0,2019-09-17 00:52:26,https://pbs.twimg.com/profile_images/145893433...,True,False,0.0,2.0
4,YESSSIR DOING A QUICK STREAM BEFOE TONIGHT'S #...,2022-01-23 23:58:56,1.485402e+18,Chris RossVlogs,ChrisRossvlogs,1.347083e+18,Mr. Hollywood,https://t.co/3xdQXI2Zv8,"Actor/Director, & Youtuber Use code “ChrisRoss...",False,...,276.0,4211.0,4099.0,1.0,2021-01-07 07:32:01,https://pbs.twimg.com/profile_images/141594587...,True,False,1.0,3.0


In [17]:
# Now that we have the tweet df containing all tweets across all weeks. I want to store that df on my local machine

df.to_csv('euphoria_tweets_complete.csv')

In [18]:
tweets_df = pd.read_csv('euphoria_tweets_complete.csv', index_col=0)

/Users/iizokun/opt/miniconda3/envs/ds102/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (10,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
tweets_df.dtypes

tweet_full_text                object
tweet_created_at               object
tweet_id                      float64
tweet_username                 object
tweet_user_screename           object
tweet_userid_str              float64
tweet_user_location            object
tweet_user_url                 object
tweet_user_description         object
tweet_user_verified            object
tweet_user_followers_count    float64
tweet_user_friends_count      float64
tweet_user_favorites_count    float64
tweet_user_statuses_count     float64
tweet_user_listed_count       float64
tweet_user_created_at          object
user_profile_image_url         object
user_default_profile           object
user_default_profile_image     object
tweet_retweet_count           float64
tweet_favorite_count          float64
dtype: object

## Duplicate Values

The `tweet_id` field is a unique string of characters that identifies that particular tweet so it'll be the field that's used to identify whether or not there are duplicates for any particular tweet. 

In [20]:
tweets_df.duplicated(subset='tweet_id').sum() # Check for duplicate values

49933

In [21]:
duplicate = tweets_df[tweets_df.duplicated(subset='tweet_id')]

In [22]:
print("Duplicate Rows :")
duplicate.head()

Duplicate Rows :


,tweet_full_text,tweet_created_at,tweet_id,tweet_username,tweet_user_screename,tweet_userid_str,tweet_user_location,tweet_user_url,tweet_user_description,tweet_user_verified,...,tweet_user_friends_count,tweet_user_favorites_count,tweet_user_statuses_count,tweet_user_listed_count,tweet_user_created_at,user_profile_image_url,user_default_profile,user_default_profile_image,tweet_retweet_count,tweet_favorite_count
1,When are we getting her backstory? #Euphoria h...,18/01/2022 23:59,1.483590e+18,pingüino stan,spacetono,1.031210e+18,he/they,NaN,"published author, film star, director, student...",False,...,296.0,94208.0,1051.0,0.0,19/08/2018 15:49,https://pbs.twimg.com/profile_images/148357692...,True,False,0.0,5.0
2,Y’all ain’t tell me Lil Meech was on #Euphoria...,18/01/2022 23:57,1.483590e+18,what you see is what you get 🤷🏾‍♀️,CenSchutt,5.914994e+07,"Miami, FL",NaN,1920 🕊 | IG: @Cen.Schutt 🇭🇹| Paralegal Bae 👩🏾‍...,False,...,389.0,3929.0,34843.0,1.0,22/07/2009 15:10,https://pbs.twimg.com/profile_images/148005706...,False,False,0.0,0.0
3,I think @sydney_sweeney is one of the most bea...,18/01/2022 23:57,1.483590e+18,danielle,dac1971,7.117740e+07,NaN,https://t.co/RZJ1yyGKiT,https://t.co/FKOSlKk1w6,False,...,97.0,41711.0,61967.0,45.0,03/09/2009 05:03,https://pbs.twimg.com/profile_images/148238200...,False,False,0.0,2.0
4,We’re watching Ethan’s villain origin before o...,18/01/2022 23:56,1.483590e+18,keturah 🪐,KeturahTheGreat,3.071322e+08,901|615,NaN,Living with ease. 🤍🕯 She/her. INFJ-T.,False,...,1612.0,217856.0,108262.0,8.0,29/05/2011 02:46,https://pbs.twimg.com/profile_images/146354440...,False,False,0.0,7.0
5,lana gonna eat on the #Euphoria song https://t...,18/01/2022 23:56,1.483590e+18,brady,1989DEFAULT,1.230300e+18,honeymoon,NaN,fan account,False,...,1326.0,7835.0,11171.0,3.0,20/02/2020 01:33,https://pbs.twimg.com/profile_images/148359064...,True,False,0.0,1.0


In [23]:
tweets_df=tweets_df.drop_duplicates(subset=['tweet_id']) # drop duplicate values


In [24]:
tweets_df.shape

(186707, 21)

## Null Values

In [25]:
tweets_df.isna().sum()

tweet_full_text                    0
tweet_created_at                   0
tweet_id                           0
tweet_username                     9
tweet_user_screename               0
tweet_userid_str                   0
tweet_user_location            62051
tweet_user_url                123593
tweet_user_description         17706
tweet_user_verified                2
tweet_user_followers_count         2
tweet_user_friends_count           2
tweet_user_favorites_count         5
tweet_user_statuses_count          5
tweet_user_listed_count            5
tweet_user_created_at              5
user_profile_image_url             7
user_default_profile               5
user_default_profile_image         5
tweet_retweet_count                5
tweet_favorite_count               5
dtype: int64

In [26]:
tweets_df.isnull().sum()

tweet_full_text                    0
tweet_created_at                   0
tweet_id                           0
tweet_username                     9
tweet_user_screename               0
tweet_userid_str                   0
tweet_user_location            62051
tweet_user_url                123593
tweet_user_description         17706
tweet_user_verified                2
tweet_user_followers_count         2
tweet_user_friends_count           2
tweet_user_favorites_count         5
tweet_user_statuses_count          5
tweet_user_listed_count            5
tweet_user_created_at              5
user_profile_image_url             7
user_default_profile               5
user_default_profile_image         5
tweet_retweet_count                5
tweet_favorite_count               5
dtype: int64

In [27]:
# Notice that the three main columns with null/na values are user location, user url, and user description. Two of these will most likely get dropped 

#Replace the NaN values in the location column with 'No Location'
tweets_df['tweet_user_location'] = tweets_df['tweet_user_location'].fillna('No Location')

In [28]:
tweets_df = tweets_df.drop(columns = ['tweet_user_url', 'tweet_user_description','user_profile_image_url', 'tweet_user_favorites_count', 'tweet_user_statuses_count', 'user_default_profile', 'user_default_profile_image','tweet_user_listed_count', 'tweet_user_created_at'])

In [29]:
tweets_df = tweets_df.dropna()

In [30]:
tweets_df.isnull().sum()

tweet_full_text               0
tweet_created_at              0
tweet_id                      0
tweet_username                0
tweet_user_screename          0
tweet_userid_str              0
tweet_user_location           0
tweet_user_verified           0
tweet_user_followers_count    0
tweet_user_friends_count      0
tweet_retweet_count           0
tweet_favorite_count          0
dtype: int64

In [31]:
tweets_df.shape

(186693, 12)

## Location Geocoding Time

HERE offers the following geocoding services:

- Discover and its search-as-you-type Autosuggest engine.
- Geocode and address Autocomplete.
- Reverse Geocode
- Lookup for direct access and Browse for a slice-and-dice access to the data.

In [32]:
# HERE url and API key
geo_url = "https://geocode.search.hereapi.com/v1/geocode"
api_key = "nFMufDOPF6pBdYPgtvywIGDp0t3I4A8iDptFekcr2v4"

In [33]:
# Test to show how API and HERE service works 
test_params = {"q": 'New York City', "apiKey":api_key}

In [35]:
test_response = requests.get(geo_url, test_params)

In [37]:
if test_response.status_code == 200:
    print(test_response.json())
    print("Country", test_response.json()['items'][0]['address']['countryName'])
else:
    print("Error", status_code)

{'items': [{'title': 'New York, NY, United States', 'id': 'here:cm:namedplace:21019301', 'resultType': 'locality', 'localityType': 'city', 'address': {'label': 'New York, NY, United States', 'countryCode': 'USA', 'countryName': 'United States', 'stateCode': 'NY', 'state': 'New York', 'county': 'New York', 'city': 'New York', 'postalCode': '10007'}, 'position': {'lat': 40.71454, 'lng': -74.00712}, 'mapView': {'west': -74.2589, 'south': 40.47742, 'east': -73.70038, 'north': 40.91762}, 'scoring': {'queryScore': 1.0, 'fieldScore': {'city': 1.0}}}]}
Country United States


In [40]:
# Now we've seen the power of the API, now it's time to apply it to our DataFrame
def getGeneralLocationData(location):
    geo_params = {"q": location  , "apiKey": api_key }
    
    response = requests.get(url = geo_url, params = geo_params)
    
    location_data = response.json()
    
    return location_data
  

In [41]:
tweets_df['general_location_data'] = tweets_df['tweet_user_location'].apply(getGeneralLocationData)

In [42]:
# Let's see what the column looks like in our dataframe 
tweets_df.head(10)

,tweet_full_text,tweet_created_at,tweet_id,tweet_username,tweet_user_screename,tweet_userid_str,tweet_user_location,tweet_user_verified,tweet_user_followers_count,tweet_user_friends_count,tweet_retweet_count,tweet_favorite_count,general_location_data
0,Really want a fez / Lexi parallel scene to when Spencer / Toby from PLL played Scrabble for the first time in the motel room and slept in the same bed. Is that just too much to ask ??? #Euphoria,2022-01-23 23:59:50,1.485402e+18,Katy,katykitty628,1.471191e+18,No Location,False,36.0,82.0,0.0,0.0,{'items': []}
1,#Euphoria I hope Maddy pulls the hair out that girl's scalp tonight,2022-01-23 23:59:33,1.485402e+18,𝐕𝐢𝐫𝐠𝐞𝐚𝐮𝐱,_njauu,9.510643e+17,here,False,1586.0,221.0,0.0,0.0,"{'items': [{'title': 'Here, Chandgad, Maharashtra, India', 'id': 'here:cm:poi:1071061874', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'Here, Chandgad, Maharashtra, India', 'countryCode': 'IND', 'countryName': 'India', 'stateCode': 'MH', 'state': 'Maharashtra', 'county': 'Kolhapur', 'city': 'Chandgad', 'district': 'Here'}, 'position': {'lat': 15.88025, 'lng': 74.18321}, 'mapView': {'west': 74.1832, 'south': 15.88012, 'east': 74.18327, 'north': 15.88039}, 'scoring': {'queryScore': 1.0, 'fieldScore': {'district': 1.0}}}, {'title': 'Here, Konjic, Federacija Bosne i Hercegovine, Bosna i Hercegovina', 'id': 'here:cm:poi:1097295711', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'Here, Konjic, Federacija Bosne i Hercegovine, Bosna i Hercegovina', 'countryCode': 'BIH', 'countryName': 'Bosna i Hercegovina', 'state': 'Federacija Bosne i Hercegovine', 'county': 'Hercegovačko-Neretvanski kanton', 'city': 'Konjic', 'district': 'Here'}, 'position': {'lat': 43.81152, 'lng': 17.80095}, 'mapView': {'west': 17.80067, 'south': 43.81136, 'east': 17.80128, 'north': 43.81165}, 'scoring': {'queryScore': 1.0, 'fieldScore': {'district': 1.0}}}, {'title': 'Here, Živinice, Federacija Bosne i Hercegovine, Bosna i Hercegovina', 'id': 'here:cm:poi:1097309377', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'Here, Živinice, Federacija Bosne i Hercegovine, Bosna i Hercegovina', 'countryCode': 'BIH', 'countryName': 'Bosna i Hercegovina', 'state': 'Federacija Bosne i Hercegovine', 'county': 'Tuzlanski kanton', 'city': 'Živinice', 'district': 'Here'}, 'position': {'lat': 44.40131, 'lng': 18.73384}, 'mapView': {'west': 18.73362, 'south': 44.40109, 'east': 18.73406, 'north': 44.40153}, 'scoring': {'queryScore': 1.0, 'fieldScore': {'district': 1.0}}}, {'title': 'Here, Patmara, Karnali, Nepal', 'id': 'here:cm:poi:1099606175', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'Here, Patmara, Karnali, Nepal', 'countryCode': 'NPL', 'countryName': 'Nepal', 'state': 'Karnali', 'county': 'Jumla', 'city': 'Patmara', 'district': 'Here'}, 'position': {'lat': 29.31291, 'lng': 82.20937}, 'mapView': {'west': 82.20862, 'south': 29.31247, 'east': 82.21012, 'north': 29.31336}, 'scoring': {'queryScore': 1.0, 'fieldScore': {'district': 1.0}}}, {'title': 'هير, برعو, توجدير, الصومال', 'id': 'here:cm:poi:1176018446', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'هير, برعو, توجدير, الصومال', 'countryCode': 'SOM', 'countryName': 'الصومال', 'state': 'توجدير', 'city': 'برعو', 'district': 'هير'}, 'position': {'lat': 8.67674, 'lng': 45.83171}, 'mapView': {'west': 45.8312, 'south': 8.67618, 'east': 45.83217, 'north': 8.67738}, 'scoring': {'queryScore': 1.0, 'fieldScore': {'district': 1.0}}}, {'title': 'Here, Mayo-Sava, Cameroun', 'id': 'here:cm:poi:1200913387', 'resultType': 'locality', 'localityType': 'district', 'address': {'label': 'Here, Mayo-Sava, Cameroun', 'countryCode': 'CMR', 'countryName': 'Cameroun', 'county': 'Mayo-Sava', 'city': 'Mayo-Sava', 'district': 'Here'}, 'position': {'lat': 10.99234, 'lng': 14.19674}, 'mapView': {'west': 14.19636, 'south': 10.99158, 'east': 14.19686, 'north': 10.99302}, 'scoring': {'queryScore': 1.0,

In [43]:
# Next step would be to extract the country name and country code from our general location data column

def getCountryName(data):
    
    for x,y in data.items():
        if len(data['items']) > 0:
            country = y[0]['address']['countryName']
        else:
            country = float('Nan')
    
        return country
    

In [44]:
def getCountryCode(data):
    
    for x,y in data.items():
        if len(data['items']) > 0:
            country_code = y[0]['address']['countryCode']
        else:
            country_code = float('Nan')
    
        return country_code

In [45]:
tweets_df['country_name'] = tweets_df['general_location_data'].apply(getCountryName)

In [46]:
tweets_df['country_code'] = tweets_df['general_location_data'].apply(getCountryCode)

In [47]:
# Sanity Check
tweets_df.sample(10)

,tweet_full_text,tweet_created_at,tweet_id,tweet_username,tweet_user_screename,tweet_userid_str,tweet_user_location,tweet_user_verified,tweet_user_followers_count,tweet_user_friends_count,tweet_retweet_count,tweet_favorite_count,general_location_data,country_name,country_code
11749,Idk how much HBO is paying @Labrinth for #Euphoria but it’s not enough. Tell that check to to STAND UP https://t.co/MRehGB3lwy,2022-02-01 22:07:25,1.488635e+18,Kae King,TheKaeKing,1.292144e+18,No Location,False,373.0,478.0,0.0,0.0,{'items': []},NaN,NaN
16071,This Malcolm X tee tho. I need it \n\n#euphoria,2022-02-15 01:39:13,1.493399e+18,Queen Mami No Swami,myeshxa,1.221516e+18,No Location,False,11565.0,295.0,0.0,21.0,{'items': []},NaN,NaN
23738,I rewatched it a SECOND time immediately after 1st but in Spanish with my dad.\nGave him the most pertinent context but he's only seen S1 Ep1 before this...\nHis eyes were glued to the screen the entire time.\n@Zendaya's dedication to this character shone brightlly.\n#EuphoriaHBOMax https://t.co/I9eDNmGPhV,2022-02-08 01:07:15,1.490855e+18,🇲🇽🌹💘Lovey-Mona💖🌹🌈,MonaLovesYa,2.117646e+08,"She/They ~ Los Angeles, CA",False,480.0,81.0,0.0,1.0,"{'items': [{'title': 'Los Angeles, CA, United States', 'id': 'here:cm:namedplace:21009653', 'resultType': 'locality', 'localityType': 'city', 'address': {'label': 'Los Angeles, CA, United States', 'countryCode': 'USA', 'countryName': 'United States', 'stateCode': 'CA', 'state': 'California', 'county': 'Los Angeles', 'city': 'Los Angeles', 'postalCode': '90012'}, 'position': {'lat': 34.05357, 'lng': -118.24545}, 'mapView': {'west': -118.66817, 'south': 33.70373, 'east': -118.15521, 'north': 34.33839}, 'scoring': {'queryScore': 0.63, 'fieldScore': {'state': 1.0, 'city': 1.0}}}]}",United States,USA
22128,"#Euphoria S2E4: I honestly thought Elliot was going to be a darker character, but there’s still time lol. So far, 1 &amp; 3 felt like S1 episodes exploring the morality of these complicated characters while 2 &amp; 4 felt thin and too melodramatic (by this show’s standards, any way) https://t.co/UiroddqCRM",2022-01-31 20:05:38,1.488242e+18,DeVaughn Taylor,_daddydisco,5.913888e+08,STL ➡️ LA,False,1489.0,1814.0,0.0,0.0,{'items': []},NaN,NaN
14522,Well damn Cal going in on the whole fam 🥴🥴 #Euphoria,2022-02-01 07:59:57,1.488422e+18,la jefa,nikkigats,4.850756e+09,Kenosha✈️Killeen,False,109.0,309.0,0.0,1.0,"{'items': [{'title': 'Killeen, TX, United States', 'id': 'here:cm:namedplace:21015285', 'resultType': 'locality', 'localityType': 'city', 'address': {'label': 'Killeen, TX, United States', 'countryCode': 'USA', 'countryName': 'United States', 'stateCode': 'TX', 'state': 'Texas', 'county': 'Bell', 'city': 'Killeen', 'postalCode': '76541'}, 'position': {'lat': 31.11689, 'lng': -97.72737}, 'mapView': {'west': -97.91378, 'south': 30.90068, 'east': -97.31514, 'north': 31.17355}, 'scoring': {'queryScore': 0.44, 'fieldScore': {'city': 0.95}}}, {'title': 'Kenosha, WI, United States', 'id': 'here:cm:namedplace:21030538', 'resultType': 'locality', 'localityType': 'city', 'address': {'label': 'Kenosha, WI, United States', 'countryCode': 'USA', 'countryName': 'United States', 'stateCode': 'WI', 'state': 'Wisconsin', 'county': 'Kenosha', 'city': 'Kenosha', 'postalCode': '53140'}, 'position': {'lat': 42.58808, 'lng': -87.82326}, 'mapView': {'west': -87.99687, 'south': 42.53887, 'east': -87.8075, 'north': 42.63946}, 'scoring': {'queryScore': 0.44, 'fieldScore': {'city': 0.95}}}]}",United States,USA
1624,Experience #Euphoria 🤤 https://t.co/YAu5aqqMfV #CBG #CBDOil #CBDpets #CBD #CBG #THC #Health #CannabisLegal #50ShadesofGreen,2022-01-30 13:29:29,1.487780e+18,50 Shades of Green,HempNewsNow1,1.452355e+18,Worldwide,False,229.0,1395.0,0.0,0.0,{'items': []},NaN,NaN
15123,She was dead crying in the hallway at the mere fact that Rue had a friend that wasn’t her 😂 #Euphoria https://t.co/XVvxSAoJYZ,2022-02-01 05:32:26,1.488385e+18,India,indiwashere,7.369519e+08,IUP,False,569.0,757.0,1.0,6.0,"{'items': [{'t

In [48]:
# Remove more irrelevant columns
tweets_df = tweets_df.drop(columns = ['general_location_data'])

In [49]:
tweets_df.columns

Index(['tweet_full_text', 'tweet_created_at', 'tweet_id', 'tweet_username',
       'tweet_user_screename', 'tweet_userid_str', 'tweet_user_location',
       'tweet_user_verified', 'tweet_user_followers_count',
       'tweet_user_friends_count', 'tweet_retweet_count',
       'tweet_favorite_count', 'country_name', 'country_code'],
      dtype='object')

In [50]:
# Exporting DF at this stage with updated columns
tweets_df.to_csv('euphoria_tweets_trimmed.csv')